In [ ]:
library(tidyr)
library(dplyr)
library(ggplot2)
library(RColorBrewer)
library(gridExtra)
library(magrittr)
library(stringr)
library(tools)
library(patchwork)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 12)

In [ ]:
get_dataframes <- function(estimator, effect, n_sims) {
    ## Cette fonction permet de fusionner plusieurs dataframes en un seul
    ## afin d'avoir le bon format pour ggplot2
    ## arguments:
    ##    estimator: estimateur sous forme de chaîne de caractères
    ##    effet: effet direct ou indirect au même format
    ##    n_sims: vecteur contenant le nombre de simulations
    pattern <- paste0("^", estimator, "_", effect, "_", "[0-9]")
    my_files <- list.files(pattern = pattern, ignore.case = TRUE)
    my_data <- lapply(my_files, read.csv)

    big_data <- bind_rows(my_data)
    keycol <- "library"
    valuecol <- "bias"
    gathercols <- colnames(big_data)

    data_long <- gather_(big_data, keycol, valuecol, gathercols)
    data_long$n <- c(rep(n_sims[1], n_sims[1]),
                     rep(n_sims[2], n_sims[2]),
                     rep(n_sims[3], n_sims[3]),
                     rep(n_sims[4], n_sims[4]),
                     rep(n_sims[5], n_sims[5]))

    return(data_long)
}

In [ ]:
plot_biases <- function(estimator, effect, n_sims, main) {
    data_long <- get_dataframes(estimator, effect, n_sims)

    ggplot(data_long, aes(y = bias, fill = library)) +
    geom_boxplot() +
    geom_hline(yintercept = 0, linetype = "dashed",
               color = "black", size = 1) +
    facet_wrap(~n, nrow = 2) +
    # guides(fill = guide_legend(ncol = 2)) +
    theme(strip.text.x = element_text(size = 12, face = "bold"),
          legend.position = c(.97, .4),
          legend.justification = c("right", "top"),
          legend.box.just = "right",
          legend.margin = margin(6, 6, 6, 6),
          legend.title = element_text(size = 13, face = "bold"),
          legend.text = element_text(size = 12)) ->
    p

    return(p)
}

In [ ]:
n_sims <- c(100, 200, 300, 400, 500)

In [ ]:
p <- plot_biases("tmle", "sde", n_sims, "TMLE sur SDE")
print(p)
ggsave("C:/Users/luis_/Downloads/tmle_sl_sde.png")

In [ ]:
p <- plot_biases("tmle", "sie", n_sims, "TMLE sur SIE")
print(p)
ggsave("C:/Users/luis_/Downloads/tmle_sl_sie.png")

In [ ]:
p <- plot_biases("tmle", "sde_posit", n_sims, "TMLE sur SDE\net problème de positivité")
print(p)
ggsave("C:/Users/luis_/Downloads/tmle_posit_sl_sde.png")

In [ ]:
p <- plot_biases("tmle", "sie_posit", n_sims, "TMLE sur SIE\net problème de positivité")
print(p)
ggsave("C:/Users/luis_/Downloads/tmle_posit_sl_sie.png")

In [ ]:
p <- plot_biases("tmle", "sde_quant", n_sims, "TMLE sur SDE\net variables baseline quantitatives")
print(p)
ggsave("C:/Users/luis_/Downloads/tmle_quant_sl_sde.png")

In [ ]:
p <- plot_biases("tmle", "sie_quant", n_sims, "TMLE sur SIE\net variables baseline quantitatives")
print(p)
ggsave("C:/Users/luis_/Downloads/tmle_quant_sl_sie.png")